In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
ls'/content/gdrive/My Drive/Thesis'

dataframe_dma_flow_1D.csv     dataframe_flow_variable_1D.csv
dataframe_dma_flow_60min.csv  dataframe_flow_variable_60min.csv
dataframe_flow_1D.csv


In [4]:
data = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv")

In [5]:
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])

In [6]:
data.rename( columns={'Unnamed: 0':'Date_time','225':'DMA_225'}, inplace=True )

In [7]:
data_DMA225 = data[['Date_time','DMA_225']]

In [ ]:
data_DMA225.describe()

,DMA_225
count,2208.000000
mean,3.562107
std,1.638267
min,0.861250
25%,1.805750
50%,3.972250
75%,4.833500
max,8.916750


In [ ]:
# Checking if there are any null values:
data_DMA225.isnull().values.any()

False

In [ ]:
# Checking if there are na values:
data_DMA225.isna().values.any()

False

In [ ]:
data_DMA225.head(3)

,Date_time,DMA_225
0,2016-04-22 00:00:00,1.13875
1,2016-04-22 01:00:00,1.16650
2,2016-04-22 02:00:00,1.16650


In [ ]:
figure = px.line(data_DMA225,x="Date_time",y="DMA_225",title="DMA_225 with slider")
figure.update_xaxes(rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=2,
                     label="2m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        )
    )
figure.show()
             

In [ ]:
type(data_DMA225)

pandas.core.frame.DataFrame

In [ ]:
data_DMA225.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date_time  2208 non-null   datetime64[ns]
 1   DMA_225    2208 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.6 KB


In [ ]:
data_DMA225.shape

(2208, 2)

In [8]:
# Train Test Split Index
size_of_trainset = 0.7
split_value = round(len(data_DMA225)* size_of_trainset)

# Split
train = data_DMA225.iloc[:split_value]
test = data_DMA225.iloc[split_value:]

print(data_DMA225.shape)
print(train.shape,test.shape)


(2208, 2)
(1546, 2) (662, 2)


In [9]:
# Normalisation training:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(train.iloc[:,1:])
print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
# Normalised dataset:
df_train = scaler.transform(train.iloc[:,1:])
df_test = scaler.transform(test.iloc[:,1:])

Min: 0.916750, Max: 8.916750


In [10]:
print(len(df_train), len(df_test))

1546 662


In [ ]:
type(df_train)

numpy.ndarray

In [ ]:
type(df_test)

numpy.ndarray

In [11]:
# Converting numpy array into matrix:
def matrix_dataset(array_dataset, timesteps=1):
 mat_X, mat_Y = [], []
 for i in range(len(array_dataset)-timesteps-1):
  Z = i+timesteps
  m = array_dataset[i:Z, 0]
  mat_X.append(m)
  n = array_dataset[Z, 0]
  mat_Y.append(n)
 return np.array(mat_X), np.array( mat_Y)

In [12]:
# Preparing the data and reshaping it as my  LSTM input layer must be 3D (2 dimensions - samples, time steps, and features)
timesteps =  10
train_X,train_y = matrix_dataset(df_train,timesteps)
test_X, test_y = matrix_dataset(df_test,timesteps)
print("The train_X and train_y values are:",train_X.shape,train_y.shape)
print("The test_X and test_y values are:",test_X.shape,test_y.shape)



The train_X and train_y values are: (1535, 10) (1535,)
The test_X and test_y values are: (651, 10) (651,)


In [13]:
#Reshaping for lstm input
#Feature should be 1 to get lstm input
timesteps = 10
feature = 1
train_X = train_X.reshape((train_X.shape[0], timesteps, feature))
test_X = test_X.reshape((test_X.shape[0], timesteps, feature))
print("The train_X values after reshaping:",train_X.shape)
print("The test_X values after reshaping:",test_X.shape)


The train_X values after reshaping: (1535, 10, 1)
The test_X values after reshaping: (651, 10, 1)


# LSTM Model

In [14]:
lstm_model=Sequential()
lstm_model.add(LSTM(200,input_shape=(timesteps,feature)))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(train_X,train_y,validation_data=(test_X,test_y),epochs=200,batch_size=200, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)
lstm_model.compile(loss='mean_squared_error',optimizer='adam')

Epoch 1/200
8/8 [==============================] - 3s 195ms/step - loss: 0.0816 - val_loss: 0.0441
Epoch 2/200
8/8 [==============================] - 1s 92ms/step - loss: 0.0401 - val_loss: 0.0369
Epoch 3/200
8/8 [==============================] - 1s 94ms/step - loss: 0.0341 - val_loss: 0.0285
Epoch 4/200
8/8 [==============================] - 1s 91ms/step - loss: 0.0277 - val_loss: 0.0244
Epoch 5/200
8/8 [==============================] - 1s 116ms/step - loss: 0.0233 - val_loss: 0.0212
Epoch 6/200
8/8 [==============================] - 1s 92ms/step - loss: 0.0194 - val_loss: 0.0176
Epoch 7/200
8/8 [==============================] - 1s 91ms/step - loss: 0.0164 - val_loss: 0.0168
Epoch 8/200
8/8 [==============================] - 1s 92ms/step - loss: 0.0157 - val_loss: 0.0168
Epoch 9/200
8/8 [==============================] - 1s 90ms/step - loss: 0.0152 - val_loss: 0.0151
Epoch 10/200
8/8 [==============================] - 1s 92ms/step - loss: 0.0143 - val_loss: 0.0144
Epoch 11/200
8/8 

In [15]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200)               161600    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Prediction and getting the accuracy metrics
predict_train=lstm_model.predict(train_X)
predict_test=lstm_model.predict(test_X)

##Transformback to original form
predict_train_lstm=scaler.inverse_transform(predict_train)
predict_test_lstm=scaler.inverse_transform(predict_test)

In [17]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset = math.sqrt(mean_squared_error(train_y,predict_train_lstm))
testset = math.sqrt(mean_squared_error(test_y,predict_test_lstm))

print("The RMSE prediction value on trainset: ",trainset)
print("The RMSE prediction value on testset: ",testset)

The RMSE prediction value on trainset:  3.5473046454850516
The RMSE prediction value on testset:  3.5971186471411665


# Stacked LSTM

In [21]:
S_LSTMmodel=Sequential()
S_LSTMmodel.add(LSTM(200,return_sequences=True,input_shape=(timesteps,feature)))
S_LSTMmodel.add(LSTM(100,return_sequences=True))
S_LSTMmodel.add(LSTM(50))
#S_LSTMmodel.add(Dropout(0.2))
#S_LSTMmodel.add(LSTM(20,return_sequences=False))
S_LSTMmodel.add(Dropout(0.5))
#S_LSTMmodel.add(Dense(20, activation='relu'))
S_LSTMmodel.add(Dense(1,activation='linear'))
S_LSTMmodel.compile(loss='mean_squared_error',optimizer='adam')
hy= S_LSTMmodel.fit(train_X,train_y,validation_data=(test_X,test_y),epochs=200,batch_size=200, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)
S_LSTMmodel.compile(loss='mean_squared_error',optimizer='adam')

Epoch 1/200
8/8 [==============================] - 7s 340ms/step - loss: 0.0997 - val_loss: 0.0502
Epoch 2/200
8/8 [==============================] - 1s 171ms/step - loss: 0.0524 - val_loss: 0.0439
Epoch 3/200
8/8 [==============================] - 1s 171ms/step - loss: 0.0458 - val_loss: 0.0399
Epoch 4/200
8/8 [==============================] - 1s 168ms/step - loss: 0.0419 - val_loss: 0.0350
Epoch 5/200
8/8 [==============================] - 1s 174ms/step - loss: 0.0356 - val_loss: 0.0337
Epoch 6/200
8/8 [==============================] - 1s 169ms/step - loss: 0.0336 - val_loss: 0.0367
Epoch 7/200
8/8 [==============================] - 1s 170ms/step - loss: 0.0334 - val_loss: 0.0303
Epoch 8/200
8/8 [==============================] - 1s 172ms/step - loss: 0.0303 - val_loss: 0.0262
Epoch 9/200
8/8 [==============================] - 1s 172ms/step - loss: 0.0267 - val_loss: 0.0258
Epoch 10/200
8/8 [==============================] - 1s 170ms/step - loss: 0.0261 - val_loss: 0.0261
Epoch 11/

In [23]:
S_LSTMmodel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 200)           161600    
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 100)           120400    
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 312,251
Trainable params: 312,251
Non-trainable params: 0
_________________________________________________________________


In [24]:
#Prediction and getting the accuracy metrics
predict_train_Slstm=S_LSTMmodel.predict(train_X)
predict_test_Slstm=S_LSTMmodel.predict(test_X)

##Transformback to original form
predict_train_Slstm=scaler.inverse_transform(predict_train_Slstm)
predict_test_Slstm=scaler.inverse_transform(predict_test_Slstm)

In [25]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset_Slstm = math.sqrt(mean_squared_error(train_y,predict_train_Slstm))
testset_Slstm = math.sqrt(mean_squared_error(test_y,predict_test_Slstm))

print("The S_lstm RMSE prediction value on trainset: ",trainset_Slstm)
print("The S_lstm RMSE prediction value on testset: ",testset_Slstm)

The S_lstm RMSE prediction value on trainset:  3.418418201940549
The S_lstm RMSE prediction value on testset:  3.457373220241675
